## Лабораторная работа №1 | Вариант 4

Выполнил: Недавний А., 311

### 1. Numpy

In [ ]:
import numpy as np

1. Найти индексы не нулевых элементов массива [1,2,0,0,4,0]

In [ ]:
arr = [1, 2, 0, 0, 4, 0]
print(np.nonzero(arr)[0])

2. Создайте единичную матрицу размером 3х3

In [ ]:
np.eye(3)

3. Создайте матрицу-шахматную доску 8х8 используя функцию `tile`

In [ ]:
np.tile([[1, 0], [0, 1]], (4, 4))

4. Дан 1D массив, удалите все элементы которые входят в диапазон `[3, 8]` inplace

In [ ]:
arr = [0, 4, 4, 2, 1, 4, 8, 7, 10, 15]
np_arr = np.array(arr)
np_arr = np_arr[(np_arr < 3) | (np_arr > 8)]

5. Создайте случайный вектор размером 10 элементов и замените максимальный элемент на 0

In [ ]:
random_vec = np.random.randint(0, 100, 10)
print (random_vec)
random_vec[np.argmax(random_vec)] = 0
print (random_vec)

6. Как распечатать все значения в массиве?

In [ ]:
np.set_printoptions(threshold=np.inf)
arr = np.arange(1000)
print(arr)

7. Рассмотрим случайный вектор с формой 100х2, представляющей координаты, и найдем расстояния от точки к точке

In [ ]:
coords = np.random.rand(100, 2)
print(np.sqrt(np.sum((coords[:, None, :] - coords[None, :, :]) ** 2, axis=-1)))

8. Как конвертировать массив float(32 bit) к массиву целых чисел (integer 32 bit) in place?

In [ ]:
float_arr = np.random.rand(10).astype(np.float32)
int_arr = float_arr.astype(np.int32, copy=False)

9. Как прочитать заданный файл?
```
1, 2, 3, 4, 5
6,  ,  , 7, 8
 ,  , 9,10,11
```

будем считать что данные лежат в файле file.txt в одной директории с ipynb

In [ ]:
data = np.genfromtxt("file.txt", delimiter=',', dtype=float, missing_values='', filling_values=np.nan)
print(data)

10. Какой эквивалент enumerate для numpy массивов?

In [ ]:
arr = np.array([1, 2, 3, 4, 10, 5, 11])
for index, value in np.ndenumerate(arr):
    ...

### 2. Pandas

Найдите ответы на вопросы или выполните действия с предложенным датасетом 

Файл: `russian-housing-market.csv`

```
price_doc - Цена продажи
id - Id транзакции
timestamp - Дата транзакции
full_sq - суммарная площадь в квадратных метрах, включая лоджию, валконы и отдельные нежилые помещения
life_sq - жилая площаль в квадратных метрах, исключая лоджии, балконы и прочие нежилые помещения
floor - для апартаментов, этаж помещения
max_floor - количество этажей в здании
material - материал стен
build_year - год постройки
num_room - количество жилых комнат
kitch_sq - площадь кухни
state - состояние апартаментов
product_type - покупка или инвестиция собственника-арендатора
sub_area - название района
```

In [ ]:
import pandas as pd

In [ ]:
data_set = pd.read_csv("russian-housing-market.csv")

1. Посчитайте количество записей в датасете с отсутствующей информацией о жилой площади жилого помещения

In [ ]:
missing_life_sq_cnt = data_set["life_sq"].isnull().sum()

2. Предложите способ заменить отсутствующие значения из пункта 1 и реализуйте его. Способ нужно аргументировать(сказать почему это достаточно хорошее приближение)

сначала посчитаем среднюю площадь жилых комнат по валидным строкам

потом среднее отношение жилой площади к общей площади по валидным строкам

для каждой строки с отсутсвующим значением life_sq проставляем значения:

-- количество комнат * средняя площадь комнаты, есди параметр num_room существует и больше нуля;

-- полная площадь * среднее отношение жилой площади к полной площади, если параметр full_sq существует и больше нуля;

-- средняя площадь по всем строкам в остальных случаях.


почему достаточно хорошее приближение:

вычисляемые данные основываются на средних значениях из уже имеющихся, поэтому не сильно изменяют глобальные средние значения

In [ ]:
missing_life_sq = data_set["life_sq"].isnull()

valid_rooms_data = data_set[data_set["life_sq"].notnull() & data_set["num_room"].notnull() & (data_set["num_room"] > 0)]
avg_room_area = (valid_rooms_data["life_sq"] / valid_rooms_data["num_room"]).mean()

valid_life_sq_data = data_set[data_set["life_sq"].notnull() & data_set["full_sq"].notnull() & (data_set["full_sq"] > 0)]
mean_ratio = (valid_life_sq_data["life_sq"] / valid_life_sq_data["full_sq"]).mean()

mean_life_sq = data_set["life_sq"].mean()

data_set.loc[missing_life_sq, "life_sq"] = data_set.loc[missing_life_sq].apply(
    lambda row:
        row["num_room"] * avg_room_area if pd.notnull(row["num_room"]) and (row["num_room"] > 0) else
        row["full_sq"] * mean_ratio if pd.isnull(row["life_sq"]) and (row["full_sq"] > 0) else
        mean_life_sq,
    axis=1
)

3. Какие значения имеет 25, 50 и 75 перцентили полной площади помещения

In [ ]:
percentiles = data_set["full_sq"].quantile([0.25, 0.5, 0.75])
print(percentiles)

4. Посчитайте среднюю стоимость продажи помещений с общей площадью от 50 до 70 квадратных метров

In [ ]:
filtered_ds = data_set[(data_set["full_sq"] >= 50) & (data_set["full_sq"] <= 70)]
avg_price = filtered_ds["price_doc"].mean()

5. Какая максимальная этажность здания представлена в датасете

In [ ]:
print(data_set["max_floor"].max())

6. Выявите район с самым дорогим жильем

In [ ]:
most_expnesive_region = data_set.groupby("sub_area")["price_doc"].mean().idxmax()
print(most_expnesive_region)

7. Посчитайте среднюю стоимость за квадратный метр жилья в Митино(Mitino)

In [ ]:
mitino_ds = data_set[data_set["sub_area"] == "Mitino"]
avg_sq_cost = (mitino_ds["price_doc"] / mitino_ds["full_sq"]).mean()

8. Посчитайте среднюю площадь комнаты в Бибирево

In [ ]:
bibirevo_ds = data_set[data_set["sub_area"] == "Bibirevo"]
avg_bibirevo_room_area = (bibirevo_ds["life_sq"] / bibirevo_ds["num_room"]).mean()

9. Проанализируйте есть ли разница в цене между квартирами на 1 и 9 этажах

In [ ]:
first_floor = data_set[data_set["floor"] == 1]["price_doc"].mean()
ninth_floor = data_set[data_set["floor"] == 9]["price_doc"].mean()
print(f"difference: {first_floor - ninth_floor}")

10. Найдите минимальную стоимость квартиры с жилой площадью не менее 60 квадратных метров, не менее чем 3мя комнатами и не ниже чем на 5 этаже

In [ ]:
filtered_ds = data_set[
    (data_set["life_sq"] >= 60) &
    (data_set["num_room"] >= 3) &
    (data_set["floor"] >= 5)
]
min_price = filtered_ds["price_doc"].min()
print(min_price)

### 3. Визуализация

Используя библиотеки matplotlib/sns/plotly/pandas и набор данных представленный ранее визуализируйте

1. Как менялась средняя цена квартиры в Нагатинском Затоне(Nagatinski Zaton) с течением времени

*в табличке `Nagatinskij Zaton`

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

nz_data = data_set[data_set["sub_area"] == "Nagatinskij Zaton"]

nz_data["timestamp"] = pd.to_datetime(nz_data["timestamp"])
avg_month_price = nz_data.groupby(nz_data["timestamp"].dt.to_period("M"))["price_doc"].mean()

def millions_formatter(x, pos):
    return f'{x * 1e-6:.1f}'

plt.figure(figsize=(14, 6))
avg_month_price.plot(kind="line")
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(millions_formatter))
plt.xlabel("месяц")
plt.ylabel("средняя стоимость, млн у.е.")
plt.show()

2. Постройте гистограмму распределения цен на квартиры с общей площадью более 50 кв.метров

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

filtered_ds = data_set[data_set["full_sq"] > 50]

def millions_formatter(x, pos):
    return f'{x * 1e-6:.1f}'

plt.figure(figsize=(10, 6))
plt.grid()
sns.histplot(filtered_ds["price_doc"], bins=30, color="blue")
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(millions_formatter))
plt.xlabel("стоимость")
plt.ylabel("количество квартир")
plt.show()


3. Визуализируйте количество проданных квартир в разных районах

фан факт: plotly багает в ipynb на версии >= 6.0 и вместо значений пишет индексы элементов

In [ ]:
import plotly.express as px

sales_by_area = data_set["sub_area"].value_counts().reset_index()
sales_by_area.columns = ["sub_area", "count"]

fig = px.bar(
    sales_by_area,
    x="sub_area",
    y="count",
    labels={"sub_area": "район", "count": "продажи"},
    height=600
)

fig.update_xaxes(
    tickangle=45,
    tickfont=dict(size=10),
    rangeslider_visible=True
)
fig.update_layout(
    xaxis={"categoryorder": "total descending"},
    margin=dict(l=50, r=50, t=50, b=100)
)

fig.show()

4. Постройте box plot отображающий распределение полных площадей квартир в Басманном районе

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

b_data = data_set[data_set["sub_area"] == "Basmannoe"]

plt.figure(figsize=(8, 6))
sns.boxplot(x=b_data["full_sq"])
plt.xlabel("full square")
plt.show()

5. Визуализируйте зависимость средней стоимости квартир в зависимости от этажа

In [ ]:
import matplotlib.pyplot as plt

avg_price_by_floor = data_set.groupby('floor')['price_doc'].mean()

def millions_formatter(x, pos):
    return f'{x * 1e-6:.1f}'

plt.figure(figsize=(12, 6))
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(millions_formatter))
avg_price_by_floor.plot(kind="line")
plt.xlabel("этаж")
plt.ylabel("средняя стоимость, млн у.е.")
plt.grid()
plt.show()

6. Покажите как меняется средняя площадь квартир в зависимости от района

In [ ]:
import plotly.express as px

avg_area_by_sub_area = data_set.groupby("sub_area")["full_sq"].mean().sort_values().reset_index()
avg_area_by_sub_area.columns = ["sub_area", "avg_sq"]

fig = px.bar(
    avg_area_by_sub_area,
    x="sub_area",
    y="avg_sq",
    labels={"avg_sq" : "средняя площадь", "sub_area" : "район"},
    height=800
)

fig.update_xaxes(
    tickangle=45,
    tickfont=dict(size=10),
    rangeslider_visible=True
)

fig.update_layout(
    yaxis={"categoryorder" : "total descending"},
)

fig.show()

7. Постройте график изменения медианной цены на квартиру менее 50 кв. метров за период с 2011 до 2014 года*

\* Задание на доп. балл

In [ ]:
filtered_data = data_set[
    (data_set["full_sq"] < 50) & 
    (data_set["timestamp"] >= "2011-01-01") & 
    (data_set["timestamp"] <= "2014-12-31")]

filtered_data["timestamp"] = pd.to_datetime(filtered_data["timestamp"])
median_price_by_month = filtered_data.groupby(filtered_data["timestamp"].dt.to_period('M'))["price_doc"].median()

def millions_formatter(x, pos):
    return f'{x * 1e-6:.1f}'

plt.figure(figsize=(14, 6))
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(millions_formatter))
median_price_by_month.plot(kind="line")
plt.xlabel("месяц")
plt.ylabel("медианная стоимость, млн у.е.")
plt.grid()
plt.show()